Download a bucket from S3 to the local filesystem.

In [1]:
import boto3, os
from timeit import default_timer as timer

In [2]:
bucket_name = 'slavs-lambda-scraper'
download_dir = './data'

In [6]:
def download_s3(client, resource, dist, local='/tmp', bucket='your_bucket'):
    paginator = client.get_paginator('list_objects')
    for result in paginator.paginate(Bucket=bucket, Delimiter='/', Prefix=dist):
        if result.get('CommonPrefixes') is not None:
            for subdir in result.get('CommonPrefixes'):
                download_s3(client, resource, subdir.get('Prefix'), local, bucket)
        if result.get('Contents') is not None:
            for file in result.get('Contents'):
                if not os.path.exists(os.path.dirname(local + os.sep + file.get('Key'))):
                     os.makedirs(os.path.dirname(local + os.sep + file.get('Key')))
                resource.meta.client.download_file(bucket, file.get('Key'), local + os.sep + file.get('Key'))
                print('.', end='')

In [4]:
s3_client = boto3.client('s3')
s3_resource = boto3.resource('s3')

In [ ]:
start = timer()
print('Processing {} '.format(bucket_name), end='')
os.makedirs(download_dir, exist_ok=True)
download_s3(s3_client, s3_resource, '', download_dir, bucket=bucket_name)
print('Done in {}s'.format(timer() - start))